In [1]:
import numpy as np
import pandas as pd
from imageio import imread
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import os
from common import *
from models import own_LinRegGD
#import cv2
#import tensorflow as tf
from models import NeuralNetwork, Layer, Fixed_layer, binary_classifier, relu

print(f"Root directory: {os.getcwd()}")
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
})

#%matplotlib

Root directory: c:\Users\andre\Dropbox\FYS-STK4155_projects\FYS-STK4155 - Project2\FYS-STK4155-Prj2_report\code


# XOR classification
The following code checks that a Fixed neural net is able to correctly classify the XOR problem

In [2]:
np.random.seed(4155)
X = np.array([[1, 0],
              [0, 1],
              [1, 1],
              [0, 0]])

hidden1 = Fixed_layer(nbf_inputs=2, nbf_outputs=2, weights=np.array([[1,1],[1,1]]), bias=np.array([-1.5, -0.5]), activation_function=binary_classifier)
output = Fixed_layer(nbf_inputs=2, nbf_outputs=1, weights=np.array([-1,1]), bias=np.array([-0.5]), activation_function=binary_classifier)

model = NeuralNetwork()
model.add_layer(hidden1)
model.add_layer(output)

y_hat = model.predict(X)
print(f"y_hat: {y_hat}")

y_hat: [1 1 0 0]


In [3]:
np.random.seed(4155)
X = np.array([[1, 0],
              [1, 1],
              [0, 0]])

X_test = np.array([0, 1])
t = np.array([1,0,0])

#hidden1 = Layer(nbf_inputs=2, nbf_outputs=4, activation_function=relu)
#hidden2 = Layer(nbf_inputs=hidden1.output, nbf_outputs=16)
hidden1 = Layer(nbf_inputs=2, nbf_outputs=2, activation_function=relu, name="hidden1")
output = Layer(nbf_inputs=hidden1.output, nbf_outputs=1, name="output")

model2 = NeuralNetwork()
model2.add_layer(hidden1)
model2.add_layer(output)

model2.fit(X, t)

print(model2.sequential_layers[-1].deltas)
print(model2.sequential_layers[-2].deltas)
#y_hat = model2.predict(X)
#model2.backward_prop(None, None)

a_out.shape: (3, 1)
t.shape: (3, 1)
output_layer.grad_activation(a_out).shape: (3, 1)
current: hidden1
right: output
current.weights.T.shape (2, 2)
right.deltas.T.shape (1, 1)
right.deltas.shape (1, 1)
backprop is done
Updating the weights

hidden weights before update
[[ 0.25437443  0.09589081]
 [ 0.48668483 -2.2087789 ]] 


right.weights before:
(2, 1)
[[-0.90144138]
 [ 0.4403852 ]]

Current is:  Layer name: hidden1

Right is:  Layer name: output

right.deltas.shape: (1, 1)

current.a.T.shape: (2, 1)

right.weights updated:
[[-0.9014306 ]
 [ 0.44038932]]

hidden weights after update
[[ 0.25437176  0.09589198]
 [ 0.48668215 -2.20877773]] 

[[-0.01498142]]
[[ 0.00267819 -0.00117127]]
